### Implementation of Modern Portfolio Theory

In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
import datetime as dt
import scipy as sc

In [2]:
# Import data
def getData(stocks, start, end):
    stockData = yf.download(stocks, start, end)
    stockData = stockData['Close']
    
    returns = stockData.pct_change()
    meanReturns = returns.mean()
    covMatrix = returns.cov()

    return meanReturns, covMatrix

def portfolio_performance(weights, meanReturns, covMatrix):
    returns = np.sum(meanReturns*weights)*252
    std = np.sqrt(np.dot(weights.T, np.dot(covMatrix, weights))) * np.sqrt(252)
    return returns, std

stockList = ['CBA', 'BHP', "TLS"]
stocks = [stock+'.AX' for stock in stockList]

endDate = dt.datetime.now()
startDate = endDate - dt.timedelta(days=365)

weights = np.array([0.3, 0.3, 0.4])

In [3]:
def negative_sharpe_ratio(weights, meanReturns, covMatrix, riskFreeRate=0):
    pReturns, pStd = portfolio_performance(weights, meanReturns, covMatrix)
    return -(pReturns - riskFreeRate)/pStd

In [4]:
def maximize_sharpe_ratio(meanReturns, covMatrix, riskFreeRate = 0, constraintSet = (0,1)):
    "Minimize the negative sharpe ratio, by altering the weights of the portfolio"
    numAssets = len(meanReturns)
    args = (meanReturns, covMatrix, riskFreeRate)
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bound = constraintSet
    bounds = tuple(bound for asset in range(numAssets))
    result = sc.optimize.minimize(negative_sharpe_ratio, numAssets*[1./numAssets], args=args, method='SLSQP', bounds=bounds, constraints=constraints)
    return result

In [5]:
meanReturns , covMatrix = getData(stocks, startDate, endDate)
returns, std = portfolio_performance(weights, meanReturns, covMatrix)
results = maximize_sharpe_ratio(meanReturns, covMatrix)
print(results)

[*********************100%%**********************]  3 of 3 completed

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: -0.8858044965712679
       x: [ 1.110e-16  1.000e+00  2.220e-16]
     nit: 2
     jac: [ 5.119e-01 -0.000e+00  1.327e+00]
    nfev: 8
    njev: 2


In [6]:
max_sharpe_ratio, maxWeights = results['fun'], results['x']
print(max_sharpe_ratio, maxWeights)

-0.8858044965712679 [1.11022302e-16 1.00000000e+00 2.22044605e-16]


In [7]:
def portfolio_variance(weights, meanReturns, covMatrix):
    return portfolio_performance(weights, meanReturns, covMatrix)[1]

In [8]:
def minimize_variance(meanReturns, covMatrix, constraintSet=(0,1)):
    "Minimize the portfolio variance by altering the weights/allocation of assets in the portfolio"
    numAssets = len(meanReturns)
    args = (meanReturns, covMatrix)
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bound = constraintSet
    bounds = tuple(bound for asset in range(numAssets))
    result = sc.optimize.minimize(portfolio_variance, numAssets*[1./numAssets], args=args, method='SLSQP', bounds=bounds, constraints=constraints)
    return result

In [9]:
minVarResult = minimize_variance(meanReturns, covMatrix)
minVar, minVarWeights = minVarResult['fun'], minVarResult['x']
print(minVar, minVarWeights)

0.11025720205262685 [0.14157835 0.28708229 0.57133936]


In [23]:
def portfolio_returns(weights, meanReturns, covMatrix):
    return portfolio_performance(weights, meanReturns, covMatrix)[0]

def efficient_optim(meanReturns, covMatrix, returnTarget, constraintSet=(0,1)):
    """
        For each return target we want to optimize the portfolio for min variance
    """
    numAssets = len(meanReturns)
    args = (meanReturns, covMatrix)
    constraints = ({'type':'eq', 'fun': lambda x: portfolio_returns(x, meanReturns, covMatrix) - returnTarget}, 
                   {'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bound = constraintSet
    bounds = tuple(bound for asset in range(numAssets))
    effOpt = sc.optimize.minimize(portfolio_variance, numAssets*[1./numAssets], args=args, 
                                  method='SLSQP', bounds=bounds, constraints=constraints)
    return effOpt

In [24]:
print(efficient_optim(meanReturns, covMatrix, returnTarget=0.1))

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 0.14069985911960903
       x: [ 8.799e-02  8.322e-01  7.983e-02]
     nit: 5
     jac: [ 1.043e-01  1.534e-01  4.828e-02]
    nfev: 20
    njev: 5


In [27]:
def calculated_results(meanReturns, covMatrix, riskFreeRate=0, constraintSet=(0,1)):
    """Read in mean, cov matrix and other financial information,
        Output Max SR, Min volatility, efficient frontier"""
    # Max Sharpe ratio portfolio
    maxSR_Portfolio = maximize_sharpe_ratio(meanReturns, covMatrix)
    maxSR_returns, maxSR_std = portfolio_performance(maxSR_Portfolio['x'], meanReturns, covMatrix)
    maxSR_returns, maxSR_std = round(maxSR_returns*100, 2), round(maxSR_std*100, 2)
    maxSR_allocation = pd.DataFrame(maxSR_Portfolio['x'], index=meanReturns.index, columns=['allocation'])
    maxSR_allocation.allocation = [round(i*100,2) for i in maxSR_allocation.allocation]
    
    # Min volatility portfolio
    minVol_Portfolio = minimize_variance(meanReturns, covMatrix)
    minVol_returns, minVol_std = portfolio_performance(minVol_Portfolio['x'], meanReturns, covMatrix)
    minVol_returns, minVol_std = round(minVol_returns*100,2), round(minVol_std*100, 2)
    minVol_allocation = pd.DataFrame(minVol_Portfolio['x'], index=meanReturns.index, columns=['allocation'])
    minVol_allocation.allocation = [round(i*100,2) for i in minVol_allocation.allocation]

    # Efficient frontier
    efficientList = []
    targetReturns = np.linspace(minVol_returns, maxSR_returns, 20)
    for target in targetReturns:
        efficientList.append(efficient_optim(meanReturns, covMatrix, target)['fun'])
    
    return maxSR_returns, maxSR_std, maxSR_allocation, minVol_returns, minVol_std, minVol_allocation, efficientList

print(calculated_results(meanReturns, covMatrix))

(13.73, 15.5,         allocation
Ticker            
BHP.AX         0.0
CBA.AX       100.0
TLS.AX         0.0, -5.92, 11.03,         allocation
Ticker            
BHP.AX       14.16
CBA.AX       28.71
TLS.AX       57.13, [0.13095130788747453, 0.13095130788934148, 0.13095130788747458, 0.13095130788747453, 0.13095130788747933, 0.13095130788747508, 0.1549595444963387, 0.1549595444963386, 0.15495954449641705, 0.15495954449632504, 0.1549595444964085, 0.15495954449632496, 0.15495954449634292, 0.15495954449632993, 0.15495954449651617, 0.15495954449632507, 0.15495954449632474, 0.15495954449632485, 0.15495954449639845, 0.15495954449714305])
